### В этом файле хранятся все полезные функции

In [2]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import os
from astropy.io import fits
import numpy as np
from matplotlib import cm

def fits_file_hist(FileName,a,b):    
    """Строит изображение из fits-файла и гистограму значений каждого пикселя
      a, b- ширина и высота картинки, Filename- путь к файлу """                                     
    data=fits.getdata(FileName)             
    data=np.nan_to_num(data)               
    width, height=data.shape[2:]        
    X,Y=np.meshgrid(np.arange(0,width,1),np.arange(0,height,1))
    data = np.array(data).reshape((width,height))
    Z=data
    fig=plt.figure(figsize=[a,b])     
    picture=plt.pcolormesh(X,Y,Z)
    plt.colorbar()
    
    FileName.replace('fits','png')
    FileName.replace('data','images')
    way=FileName
    
    plt.savefig(way,format='png')
    plt.show()
    
def fits_file(FileName,a,b):
    """Строит изображение из fits-файла"""
    data=fits.getdata(FileName)
    fig=plt.figure(figsize=[a,b])
    
   # FileName.replace('fits','png')
   # FileName.replace('data','images')
    #way=FileName
    
    plt.imshow(np.squeeze(data))
   # plt.savefig(way,format='png')
    plt.show()


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


## Функции непосредственно связанные с подсчётом параметра Тумре

In [3]:
import math
G = 4.32

### Одножидкостный критерий

In [4]:
def Qs( epic_freq = None, disp_r = None, dens_star = None ):
    return epic_freq * disp_r / math.pi / G / dens_star

In [5]:
def Qg( epic_freq = None, vel_gas = None, dens_gas = None ):
    return epic_freq * vel_gas / math.pi / G / dens_gas

### Двухжидкостный критерий

#### Гидродиномическое приближение

$$\frac{1}{Q_{\mathrm{eff}}}=\frac{2}{Q_{\mathrm{s}}}\frac{\bar{k}}{1+\bar{k}^{2}}+\frac{2}{Q_{\mathrm{g}}}s\frac{\bar{k}}{1+\bar{k}^{2}s^{2}}>1$$ для безразмерного волнового числа ${\displaystyle \bar{k}\equiv\frac{k\,\sigma_{\mathrm{s}}}{\kappa}},\, s=c/\sigma$

In [6]:
def QeffHydro(Qs = None, wave_num = None, Qg = None, s = None):
    return 1 /( (2*wave_num) / (Qs*(1+wave_num**2))  +  (2*s*wave_num) / (Qg*(1+(wave_num**2)*(s**2))) )

In [7]:
def DerivQeffHydro(k = None, Qs = None, Qg = None, s = None):
    '''Функция для подсчёта значения производной Qeff в данной точке'''
    part1 = (1 - k ** 2) / (1 + k ** 2) ** 2
    part2 = (1 - (k * s) ** 2) / (1 + (k * s) ** 2) ** 2
    part3 = QeffHydro(Qs = Qs, wave_num = k, Qg = Qg, s = s) 
    return ( (2 * part1 / Qs) + (2 * s * part2 / Qg) ) * (-1) * part3**2 
def FindMaxQeffHydro( rangek = None, Qs = None, Qg = None, s = None):
    from scipy.optimize import brentq
    '''Функция для нахождения максимума Qeff для гидродинамического приближения '''
    signs = [DerivQeffHydro(k = x, Qs = Qs, Qg = Qg, s = s) for x in rangek]
    signs = [x/abs(x) for x in signs]
   
    roots=[]
    Qeff_max=[]
    for i in range(len(signs)-1):
        if signs[i]*signs[i+1] < 0:
            roots.append(brentq(lambda x: DerivQeffHydro(k = x, Qs = Qs, Qg = Qg, s = s), rangek[i], rangek[i + 1]))
    Qeff_max=[QeffHydro(Qs = Qs, wave_num = x, Qg = Qg, s = s) for x in roots ]
    root_max=roots.index(max(Qeff_max))
    return root_max, max(Qeff_max)

#### Кинетическое приближение

$$\frac{1}{Q_{\mathrm{eff}}}=\frac{2}{Q_{\mathrm{s}}}\frac{1}{\bar{k}}\left[1-e^{-\bar{k}^{2}}I_{0}(\bar{k}^{2})\right]+\frac{2}{Q_{\mathrm{g}}}s\frac{\bar{k}}{1+\bar{k}^{2}s^{2}}>1\,$$


In [9]:
def QeffKinem(Qs = None, wave_num = None, Qg = None, s = None):
    from scipy.special import i0e
    return 1 / (2 * (1 - exp(-wave_num**(-2)) * i0e(wave_num**(-2))) / (Qs * wave_num)  +  (2 * s * wave_num) / (Qg * (1+wave_num**2*s**2)))

In [10]:
def DerivQeffKinem(k = None, Qs = None, Qg = None, s = None):
    '''Функция для подсчёта значения производной Qeff в данной точке'''
    from scipy.special import i1e, i0e
    part1 = -2 * (1 - exp(-k**(-2)) * i0e(k**(-2))) / Qs / k**2
    part2 = 2 * (1 + 2*exp(-k**(-2)) * (k*i0e(k**(-2)) + i1e(k**(-2))/k)) / Qs / k
    part3 = 2 * s * (1 - (k*s)**2) / (1 + (k*s)**2)**2 / Qg
    part4 = QeffKinem(Qs = Qs, wave_num = k, Qg = Qg, s = s)
    return (part1 + part2 + part3) * (-1) * part4**2

def FindMaxQeffKinem( rangek = None, Qs = None, Qg = None, s = None):
    from scipy.optimize import brentq
    '''Функция для нахождения максимума Qeff для кинетического приближения '''
    signs = [DerivQeffKinem(k = x, Qs = Qs, Qg = Qg, s = s) for x in rangek]
    signs = [x/abs(x) for x in signs]
   
    roots=[]
    Qeff_max=[]
    for i in range(len(signs)-1):
        if signs[i]*signs[i+1] < 0:
            roots.append(brentq(lambda x: DerivQeffKinem(k = x, Qs = Qs, Qg = Qg, s = s), rangek[i], rangek[i + 1]))
    Qeff_max=[QeffKinem(Qs = Qs, wave_num = x, Qg = Qg, s = s) for x in roots ]
    root_max=roots.index(max(Qeff_max))
    return root_max, max(Qeff_max)

